In [7]:
from __future__ import print_function, division
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

trainingdata = pd.read_csv('data/train.csv')
n = 1
id = trainingdata.iloc[n, 0]
str = trainingdata.iloc[n, 1]
hyp = trainingdata.iloc[n, 2]

print('ID: {}'.format(id))
print('sentences: {}'.format(str))


ID: 5b72532a0b
sentences: These are issues that we wrestle with in practice groups of law firms, she said. 


In [30]:
class ContradictionsDataset(Dataset):
    """contradictions dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.contradictions_set = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.contradictions_set)

    def __getitem__(self, idx):
        id = self.contradictions_set.iloc[idx, 0]
        prem = self.contradictions_set.iloc[idx, 1]
        hyp = self.contradictions_set.iloc[idx, 2:]
        lang = self.contradictions_set.iloc[idx, 3]
        label = self.contradictions_set.iloc[idx, 5]
        sample = {'premesis': prem, 'hypothesis': hyp}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
class ContradictionsTestData(Dataset):
    """contradictions test data."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.test_set = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.test_set)

    def __getitem__(self, idx):
        id = self.test_set.iloc[idx, 0]
        prem = self.test_set.iloc[idx, 1]
        hyp = self.test_set.iloc[idx, 2:]
        lang = self.test_set.iloc[idx, 3]
        #label = self.test_set.iloc[idx, 4]
        sample = {'premesis': prem, 'hypothesis': hyp}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [31]:
contradictions_dataset = ContradictionsDataset(csv_file='data/train.csv',
                                    root_dir='data/')

for i in range(len(contradictions_dataset)):
    sample = contradictions_dataset[i]

    print(i, sample)

    if i == 3:
        break

contradictions_test_data = ContradictionsTestData(csv_file='data/test.csv',
                                    root_dir='data/')

for i in range(len(contradictions_test_data)):
    sample = contradictions_test_data[i]

    print(i, sample)

    if i == 3:
        break

0 {'premesis': 'and these comments were considered in formulating the interim rules.', 'hypothesis': hypothesis    The rules developed in the interim were put to...
lang_abv                                                     en
language                                                English
label                                                         0
Name: 0, dtype: object}
1 {'premesis': 'These are issues that we wrestle with in practice groups of law firms, she said. ', 'hypothesis': hypothesis    Practice groups are not permitted to work on t...
lang_abv                                                     en
language                                                English
label                                                         2
Name: 1, dtype: object}
2 {'premesis': "Des petites choses comme celles-là font une différence énorme dans ce que j'essaye de faire.", 'hypothesis': hypothesis    J'essayais d'accomplir quelque chose.
lang_abv                                        

In [13]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = iter(contradictions_dataset)

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [14]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [15]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = contradictions_dataset
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [17]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [18]:
train_iter = contradictions_dataset
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [19]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = contradictions_dataset
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

ValueError: too many values to unpack (expected 2)